In [ ]:
#ADDING NEEDED LIBRARIES

In [ ]:
from os import listdir
from pickle import dump
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
from pickle import load
from numpy import array
import tensorflow
from pickle import load
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense  
from keras.layers import Embedding
from keras.layers import Dropout
from keras.layers.merge import add
from keras.callbacks import ModelCheckpoint
import re
import string

In [ ]:
def extract_features(directory):
  VGG_model = VGG16()
  VGG_model = Model(inputs= VGG_model.inputs, outputs=VGG_model.layers[-2].output)
  print(VGG_model.summary())
  attribute = dict()
  for i in listdir(directory):
    file_name = directory + '/' + i
    image = load_img(file_name, target_size=(224, 224))
    image = img_to_array(image)
    image = image.reshape((1, 224, 224, 3))
    image = preprocess_input(image)
    attributes = VGG_model.predict(image)
    image_id = file.split('.')[0]
    attribute[image_id] = attributes 
  return attribute

In [ ]:
directory = '/content/gdrive/MyDrive/image_caption/Flicker8k_Dataset'
features = extract_features(directory)
print('Extracted Features: %d' % len(attribute))
dump(attribute, open('features.pkl', 'wb'))

In [ ]:
def load_doc(file_name):
  file = open(file_name, 'r')
  caption = file.read()
  file.close()
  return caption

In [ ]:
def load_descriptions(document):
  map = dict()
  # process lines
  for line in document.split('n'):
    tokens = line.split()

In [ ]:
if len(line) < 2:
      continue
    image_id, image_description = tokens[0], tokens[1:]
    # removing filename from image id
    image_id = image_id.split('.')[0]
    # converting the description tokens back to string
    image_description = ' '.join(image_description)
    # create the list if needed
    if image_id not in map:
      map[image_id] = list()
    # store description
    map[image_id].append(image_description)
  return map

In [ ]:
def clean_descriptions(desc):
  table = str.maketrans('', '', string.punctuation)
  for key, desc_list in desc.items():
    for i in range(len(desc_list)):
      description = desc_list[i]
      # tokenize
      description = description.split()
      # converting the text to lower case
      description = [term.lower() for term in description]
      # removing the punctuation from each token
      description = [w.translate(table) for w in description]
      # removing the hanging 's' and 'a'
      description = [term for term in description if len(term)>1]
      # removing the tokens with numbers in them
      description = [termfor term in description if term.isalpha()]
      # storing it as astring
      desc_list[i] =  ' '.join(description)
        
def to_vocabulary(desc):
  all_desc = set()
  for key in desc.keys():
    [all_desc.update(d.split()) for d in desc[key]]
  return all_desc

def save_descriptions(desc, file_name):
  lines = list()
  for key, desc_list in desc.items():
    for description in desc_list:
      lines.append(key + ' ' + description)
  data = 'n'.join(lines)
  file = open(file_name, 'w')
  file.write(data)
  file.close()

filename = '/content/gdrive/MyDrive/image_caption/Flickr8k.token.txt'
# load descriptions
doc = load_doc(filename)
# parse descriptions
descriptions = load_descriptions(doc)
print('Loaded: %d ' % len(descriptions))
# clean descriptions
clean_descriptions(descriptions)
# summarize vocabulary
vocabulary = to_vocabulary(descriptions)
print('Vocabulary Size: %d' % len(vocabulary))
# save to file
save_descriptions(descriptions, 'descriptions.txt')

In [ ]:
def load_doc(file_name):
  file = open(file_name, 'r')
  text = file.read()
  file.close()
  return text
def load_set(file_name):
  document = load_doc(file_name)
  data_set = list()
  # process line by line
  for line in document.split('n'):
    # skip empty lines
    if len(line) < 1:
      continue
    # get the image identifier
    identifier = line.split('.')[0]
    data_set.append(identifier)
  return set(data_set)

In [ ]:
# load clean descriptions into memory
def load_clean_descriptions(file_name, data_set):
  # load document
  doc = load_doc(file_name)
  desc = dict()
  for line in doc.split('n'):
    # split line by white space
    tokens = line.split()
    # split id from description
    image_id, image_desc = tokens[0], tokens[1:]
    if image_id in data_set:
      # create list
      if image_id not in descriptions:
        desc[image_id] = list()
      # wrap desc in tokens
      description = 'startseq ' + ' '.join(image_desc) + ' endseq'
      # store
      desc[image_id].append(description)
  return desc

In [ ]:
# load photo features
def load_photo_features(file_name, data_set):
  # load all features
  all_features = load(open(file_name, 'rb'))
  # filter features
  features = {k: all_features[k] for k in data_set}
  return features

In [ ]:
def to_lines(desc):
  all_description = list()
  for key in desc.keys():
    [all_description.append(d) for d in desc[key]]
  return all_description
def create_tokenizer(desc):
  lines = to_lines(desc)
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(lines)
  return tokenizer
def max_length(desc):
  lines = to_lines(desc)
  return max(len(d.split()) for d in lines)

In [ ]:
filename = '/content/gdrive/MyDrive/image_caption/Flickr_8k.trainImages.txt'
train = load_set(filename)
print('Dataset: %d' % len(train))
# descriptions
train_descriptions = load_clean_descriptions('/content/gdrive/MyDrive/image_caption/descriptions.txt', train)
print('Descriptions: train=%d' % len(train_descriptions))
# photo features
train_features = load_photo_features('/content/gdrive/MyDrive/image_caption/features.pkl', train)
print('Photos: train=%d' % len(train_features))
# prepare tokenizer
tokenizer = create_tokenizer(train_descriptions)
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)
# determine the maximum sequence length
max_length = max_length(train_descriptions)
print('Description Length: %d' % max_length)

# create sequences of images, input sequences and output words for an image
def create_sequences(tokenizer, max_length, desc_list, photo):
  x_1 = []
  x_2 = []
  y = []
  for description in desc_list:
    # encode the sequence
    sequence = tokenizer.texts_to_sequences([description])[0]
    # spliting one sequence into multiple x and y pairs
    for i in range(1, len(sequence)):
      # split into input and output pair
      input_seq, output_seq = sequence[:i], sequence[i]
      # pad input sequence    
      in_seq = pad_sequences([input_seq], maxlen=max_length)[0]
        

In [ ]:
# encode output sequence
      out_seq = to_categorical([output_seq], num_classes=vocab_size)[0]
      # store
      x_1.append(photo)
      x_2.append(input_seq)
      y.append(output_seq)
  return array(x_1), array(x_2), array(y)
#Below code is used to progressively load the batch of data
# data generator, will be used in model.fit_generator()
def data_generator(desc, photos, tokenizer, max_length):
  # loop for ever over images
  while 1:
    for key, desc_list in desc.items():
      # retrieving the photo features
      photo = photos[key][0]
      input_img, input_seq, output_word = create_sequences(tokenizer, max_length, description_list, photo)
      yield [[input_img, input_seq], output_word]

In [ ]:
for key, desc_list in descriptions.items():
  print(desc_list)
for key, desc_list in descriptions.items(): #1st loop)
  for desc in desc_list:               #(2nd loop)
    print(desc)

for desc in desc_list:
    # encode the sequence
    seq = tokenizer.texts_to_sequences([desc])[0]

In [ ]:
for key, desc_list in descriptions.items():
  for desc in desc_list:
    seq = tokenizer.texts_to_sequences([desc])[0]
    print(seq)
    for i in range(1, len(seq)):
      in_seq, out_seq = seq[:i], seq[i]
      print(in_seq,out_seq)

In [ ]:
#MODEL BUILDING

In [ ]:
# define the captioning model
from keras.layers import LSTM
def define_model(vocab_size, max_length):
  # feature extractor model
  inputs_1 = Input(shape=(4096,))
  fe_1 = Dropout(0.5)(inputs_1)
  fe_2 = Dense(256, activation='relu')(fe_1)
  # sequence model
  inputs_2 = Input(shape=(max_length,))
  se_1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
  se_2 = Dropout(0.5)(se_1)
  se_3 = LSTM(256)(se_2)
  # decoder model
  decoder_1 = add([fe_2, se_3])
  decoder_2 = Dense(256, activation='relu')(decoder_1)
  outputs = Dense(vocab_size, activation='softmax')(decoder_2)
  # tie it together [image, sequence] [word]
  model = Model(inputs=[inputs1, inputs2], outputs=outputs)
  model.compile(loss='categorical_crossentropy', optimizer='adam')
  # summarize model
  print(model.summary())
  return model

In [ ]:
# train the model
model = define_model(vocab_size, max_length)
epochs = 20
steps = len(train_descriptions)
for i in range(epochs):
  # create the data generator
  generator = data_generator(train_descriptions, train_features, tokenizer, max_length)
  # fit for one epoch
  model.fit_generator(generator, epochs=1, steps_per_epoch=steps, verbose=1)
  # save model
  model.save('model_' + str(i) + '.h5')

In [ ]:
# map an integer to a word
def word_for_id(integer, tokenizer):
  for word, index in tokenizer.word_index.items():
    if index == integer:
      return word
  return None

In [ ]:
# generate a description for an image
def generate_desc(model, tokenizer, photo, max_length):
  # seed the generation process
  in_text = 'startseq'
  for i in range(max_length):
    # integer encode input sequence
    sequence = tokenizer.texts_to_sequences([in_text])[0]
    # pad input
    sequence = pad_sequences([sequence], maxlen=max_length)
    # predict next word
    yhat = model.predict([photo,sequence], verbose=0)
    # convert probability to integer
    yhat = argmax(yhat)
    # map integer to word
    word = word_for_id(yhat, tokenizer)
    if word is None:
      break
    # append as input for generating the next word
    in_text += ' ' + word
    # we will stop if we predict the endseq
    if word == 'endseq':
      break
  return in_text

In [ ]:
#RESULT EVALUATION

In [ ]:
# evaluate the skill of the model
def evaluate_model(model, descriptions, photos, tokenizer, max_length):
  actual, predicted = list(), list()
  # step over the whole set
  for key, desc_list in descriptions.items():
    # generate description
    yhat = generate_desc(model, tokenizer, photos[key], max_length)
    # store actual and predicted
    references = [d.split() for d in desc_list]
    actual.append(references)
    predicted.append(yhat.split())
  # calculate BLEU score
  print('BLEU-1: %f' % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
  print('BLEU-2: %f' % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))
  print('BLEU-3: %f' % corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0)))
  print('BLEU-4: %f' % corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25)))

In [ ]:
# load test set
filename = 'Flickr_8k.testImages.txt'
test = load_set(filename)
print('Dataset: %d' % len(test))
# descriptions
test_descriptions = load_clean_descriptions('descriptions.txt', test)
print('Descriptions: test=%d' % len(test_descriptions))
# photo features
test_features = load_photo_features('features.pkl', test)
print('Photos: test=%d' % len(test_features))
# load the model which has minimum loss, in this case it was model_18
filename = 'model_18.h5'
model = load_model(filename)
# evaluate model
evaluate_model(model, test_descriptions, test_features, tokenizer, max_length)